In [9]:
import os
import pandas as pd

# Adjust display options to show all columns
pd.set_option('display.max_columns', 5)
pd.set_option('display.expand_frame_repr', False)

# Relative path to the dataset directory
dataset_directory = 'Dataset'

# Get the current working directory (notebook's directory)
notebook_directory = os.getcwd()

# Construct full path to the dataset directory
directory = os.path.join(notebook_directory, dataset_directory)

# Check if the dataset directory exists
if not os.path.exists(directory):
    print(f"Error: Directory '{directory}' does not exist.")
else:
    # List all files in the dataset directory, ignoring .gitignore
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f != '.gitignore']

    # Dictionary to store DataFrames
    dfs = {}

    # Function to display loaded DataFrames
    def show_loaded_dfs(df_names=None):
        print("Currently loaded DataFrames:")
        if df_names is None:
            for name, df in dfs.items():
                print(f"DataFrame for file '{name}':")
                print(df.head())
                print("\n")
        else:
            for name in df_names:
                if name in dfs:
                    print(f"DataFrame for file '{name}':")
                    print(dfs[name].head())
                    print("\n")
                else:
                    print(f"DataFrame '{name}' not found in the loaded DataFrames.\n")

    # Function to preprocess DataFrames by dropping empty columns
    def preprocess_df(df):
        return df.dropna(axis=1, how='all')

    # Function to handle and print bad lines
    def handle_bad_line(line):
        print(f"Bad line encountered: {line}")
        return None  # Skip the bad line

    # Load each file into a DataFrame, preprocess, and create dynamic variables
    for file in files:
        file_name = os.path.splitext(file)[0]
        file_path = os.path.join(directory, file)
        try:
            # Read the file with a tab delimiter and handle bad lines using the python engine
            df = pd.read_csv(file_path, delimiter='\t', on_bad_lines=handle_bad_line, engine='python')
            if df is not None:
                df = preprocess_df(df)  # Preprocess the DataFrame
                dfs[file_name] = df
                globals()[file_name] = df  # Create a dynamic variable in the global namespace
        except pd.errors.ParserError as e:
            print(f"ParserError: {e} occurred while processing file '{file}'. Skipping this file.")
        except Exception as e:
            print(f"An error occurred while processing file '{file}': {e}")


In [8]:
# Show loaded DataFrames
show_loaded_dfs()

Currently loaded DataFrames:
DataFrame for file 'fr.openfoodfacts.org.products':
            code                                                url  ... nutrition-score-fr_100g nutrition-score-uk_100g
0  0000000003087  http://world-fr.openfoodfacts.org/produit/0000...  ...                     NaN                     NaN
1  0000000004530  http://world-fr.openfoodfacts.org/produit/0000...  ...                    14.0                    14.0
2  0000000004559  http://world-fr.openfoodfacts.org/produit/0000...  ...                     0.0                     0.0
3  0000000016087  http://world-fr.openfoodfacts.org/produit/0000...  ...                    12.0                    12.0
4  0000000016094  http://world-fr.openfoodfacts.org/produit/0000...  ...                     NaN                     NaN

[5 rows x 146 columns]




In [ ]:
# Adjust display options to show all columns
pd.set_option('display.max_columns', 10)
pd.set_option('display.expand_frame_repr', False)

def analyze_column(col):
    col_data = col.dropna()
    col_dtypes = col.dtypes
    
    if col_data.empty:
        col_type = 'NaN'
        fill_percentage = 0.0
        nan_percentage = 100.0
        bad_null_percentage = 0.0
    else:
        type_counts = col_data.apply(lambda x: type(x).__name__).value_counts(normalize=True) * 100
        if len(type_counts) == 1:
            if type_counts.index[0] != 'NaN':
                max_length = col_data.apply(lambda x: len(str(x))).max()
                col_type = f"{type_counts.index[0]}({max_length})"
            else:
                col_type = type_counts.index[0]
        else:
            # If multiple types are present, compute errorType with percentages
            error_type_details = ', '.join([f"{t}: {p:.2f}%" for t, p in type_counts.items()])
            col_type = f"errorType({error_type_details})"
        
        fill_percentage = col_data.size / col.size * 100
        nan_percentage = col.isna().sum() / col.size * 100
        
        # Check for other forms of null values
        bad_null_count = col.isin(['', 'None', 'NULL', 'null']).sum()
        bad_null_percentage = bad_null_count / col.size * 100

    return {
        'Column Name': col.name,
        'Dtype': col_dtypes,
        'Type': col_type,
        'Fill Percentage': fill_percentage,
        'NaN Percentage': nan_percentage,
        'Bad Null Percentage': bad_null_percentage
    }

def analyze_dataframe(df):
    columns_info = []
    num_rows = len(df)
    for col_name in df.columns:
        col_info = analyze_column(df[col_name])
        columns_info.append(col_info)
    df_info = pd.DataFrame(columns_info)
    return df_info, num_rows

def create_metadata_dfs(dfs):
    metadata_dfs = {}
    for df_name, df in dfs.items():
        metadata_df, num_rows = analyze_dataframe(df)
        metadata_dfs[f'metadata_{df_name} {df.shape}'] = metadata_df
    return metadata_dfs

# Function to display metadata DataFrames
def display_metadata_dfs(metadata_dfs):
    for name, metadata_df in metadata_dfs.items():
        print(f"Metadata for {name}:")
        print(metadata_df)
        print("\n")

# Analyze DataFrames and create metadata DataFrames
metadata_dfs = create_metadata_dfs(dfs)



# Save the combined metadata DataFrame to a CSV file
combined_metadata = pd.concat(metadata_dfs.values(), keys=metadata_dfs.keys()).reset_index(level=0).rename(columns={'level_0': 'DataFrame'})
combined_metadata.to_csv('data/combined_metadata.csv', index=False)

In [ ]:
# Display the metadata DataFrames
display_metadata_dfs(metadata_dfs)

In [ ]:
def check_duplicates(dfs, ignore_fields={}, mandatory_fields={}):
    """
    Checks for duplicate rows in the DataFrames and if no raw duplicates are found, checks for composite key duplicates.
    
    Parameters:
    - dfs (dict): Dictionary of DataFrames to check.
    - ignore_fields (dict): Dictionary where keys are DataFrame names and values are lists of column names to ignore.
    - mandatory_fields (dict): Dictionary where keys are DataFrame names and values are lists of column names to use as composite keys.
    
    Returns:
    - result (dict): Dictionary where keys are DataFrame names and values are tuples containing the number of raw duplicate rows and composite key duplicate rows.
    """
    result = {}
    
    for df_name, df in dfs.items():
        if df_name in ignore_fields:
            # Drop the specified columns to ignore
            df_to_check = df.drop(columns=ignore_fields[df_name], errors='ignore')
        else:
            df_to_check = df
        
        # Find raw duplicates
        duplicate_rows = df_to_check.duplicated(keep=False)
        num_raw_duplicates = duplicate_rows.sum()
        
        # Check for composite key duplicates if no raw duplicates are found
        num_composite_key_duplicates = 0
        if num_raw_duplicates == 0 and df_name in mandatory_fields:
            composite_key_columns = mandatory_fields[df_name]
            if set(composite_key_columns).issubset(df.columns):
                composite_key_duplicates = df.duplicated(subset=composite_key_columns, keep=False)
                num_composite_key_duplicates = composite_key_duplicates.sum()
        
        # Add to result
        result[df_name] = (num_raw_duplicates, num_composite_key_duplicates)
        
        print(f"DataFrame '{df_name}': {num_raw_duplicates} raw duplicate rows found, {num_composite_key_duplicates} composite key duplicate rows found")
    
    return result

# Example usage
ignore_fields = {
    'EdStatsCountry-Series': ['DESCRIPTION'],
    'EdStatsCountry': ['Special Notes'],
    'EdStatsFootNote': ['DESCRIPTION'],
    'EdStatsSeries':['Short definition']
}

mandatory_fields = {
    'EdStatsCountry-Series': ['CountryCode','SeriesCode'],
    'EdStatsCountry': ['Country Code','Table Name'],
    'EdStatsData': ['Country Code','Indicator Code'],
    'EdStatsFootNote':['CountryCode','SeriesCode','Year'],
    'EdStatsSeries':['Series Code','Indicator Name']
}

In [ ]:
# Assuming 'dfs' is the dictionary of DataFrames already loaded
duplicate_summary = check_duplicates(dfs, ignore_fields, mandatory_fields)